In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import numpy as np
from classLib_yash_Copy1 import *
from shapely.geometry import Polygon, Point
from shapely.ops import transform
from functools import partial
import pyproj

In [2]:
Path.cwd()

WindowsPath('e:/ATMRI/CD_R/ATCO_preferences_interface/Scenario_interface_work/python-code')

In [7]:
os.chdir('e:/ATMRI/CD_R/ATCO_preferences_interface/Scenario_interface_work')
airway = pd.read_json('airways.json')
airway.head(3),len(airways)

(   name                                           segments
 0  A224  [{'name': 'JOHOR BAHRU DVOR/DME (VJB)', 'lat':...
 1  A457  [{'name': 'JOHOR BAHRU DVOR/DME (VJB)', 'lat':...
 2  A464  [{'name': 'ARAMA (50 DME SJ)(Delegated airspac...,
 62)

In [9]:
def filter_airways(x1,y1,x2,y2,x3,y3,x4,y4,airway):
    filtered_airways = pd.DataFrame()

    boundary_coordinates = ([x1,y1],[x2,y2],[x3,y3],[x4,y4])

    poly = Polygon(boundary_coordinates)
    for i in range(len(airway)):
        for j in range(len(airway.segments[i])):
            point1 = Point([float(airway.segments[i][j]['lat']),float(airway.segments[i][j]['lon'])])
        # point2 = Point([float(airway.segments[i][1]['lat']),float(airway.segments[i][1]['lon'])])
            if poly.contains(point1)== True :
            # and poly.contains(point2)== True
                filtered_airways=filtered_airways.append(airway.loc[i])
    filtered_airways = filtered_airways.drop_duplicates(subset=['name'])
    return filtered_airways

# filteraws = filter_airways(1.19,103.06,1.19,104.7,1.76,104.7,1.76,103.06,airway)
# filteraws = filter_airways(0,105,0,109,5,109,5,105,airway)
filteraws = filter_airways(1,104,1,108,5,108,5,104,airway)


filteraws=filteraws.reset_index(drop=True)
# filteraws

In [10]:
# a= np.array([1.19,103.06])
a=np.array([1,104])
a_ = np.array([a[1],a[0]])
# b=  np.array([1.19,104.7])
b=  np.array([1,108])
b_ = np.array([b[1],b[0]])
# c =np.array([1.76,104.7])
c =np.array([5,104])
c_ = np.array([c[1],c[0]])
# d=  np.array([1.76,103.06])
d=  np.array([5,106])
d_ = np.array([d[1],d[0]])

horizontal_len= b[1]-a[1]
vertical_len = d[0]-a[0]

dict1=[]

for i in range(len(filteraws)):
    print(i)
    d1= []
    for j in range(len(filteraws.segments[i])):
        # print(j)
        P = np.array([float(filteraws.segments[i][j]['lon']),float(filteraws.segments[i][j]['lat'])])
        print(P)
        print("^^")
        # vert_d = np.linalg.norm(np.cross(b_-a_,P-a_))/np.linalg.norm(b_-a_)
        # horiz_d = np.linalg.norm(np.cross(d_-a_,P-a_))/np.linalg.norm(d_-a_)
        vert_d = P[1]-a_[1]
        horiz_d = P[0] -a_[0]
        airdict1={
                # "name":filteraws.name[i],        
                # "segments":[{"name":filteraws.segments[i][j]['name'],
                "name":filteraws.segments[i][j]['name'],
                "lat":filteraws.segments[i][j]['lat'],
                "lon":filteraws.segments[i][j]['lon'],
                "latdms":filteraws.segments[i][j]['latdms'],
                "londms":filteraws.segments[i][j]['londms'],
                "transf_x": (900/horizontal_len)*horiz_d,
                "transf_y": (900/vertical_len)*vert_d
                }
        d1.append(airdict1)
    # dict2={"segments":d1}
    dict3 ={"name":filteraws.name[i], 
            "segments":d1}
    # print(dict3)
    dict1.append(dict3)
        
print(dict1)

', 'lat': '4.811', 'lon': '105.380', 'latdms': '044841N', 'londms': '1052247E', 'transf_x': 310.499999999999, 'transf_y': 857.475}, {'name': 'NOPAT', 'lat': '4.387', 'lon': '104.799', 'latdms': '042313N', 'londms': '1044756E', 'transf_x': 179.77500000000148, 'transf_y': 762.0749999999999}, {'name': 'VEPLI', 'lat': '3.873', 'lon': '104.095', 'latdms': '035223N', 'londms': '1040542E', 'transf_x': 21.374999999999744, 'transf_y': 646.4250000000001}, {'name': 'BUVAL (WSJC/WMFC FIR BDRY)', 'lat': '3.606', 'lon': '103.728', 'latdms': '033622N', 'londms': '1034341E', 'transf_x': -61.20000000000125, 'transf_y': 586.35}, {'name': 'PEKAN DVOR/DME (VPK)', 'lat': '3.383', 'lon': '103.423', 'latdms': '032259N', 'londms': '1032524E', 'transf_x': -129.8249999999996, 'transf_y': 536.175}]}, {'name': 'L635', 'segments': [{'name': 'MABLI', 'lat': '4.288', 'lon': '106.213', 'latdms': '041717N', 'londms': '1061247E', 'transf_x': 497.9249999999986, 'transf_y': 739.8000000000001}, {'name': 'SUSAR', 'lat': '3

In [13]:
import json
with open('python-code/14Test1.json', 'w') as json_file:
    json.dump(dict1, json_file)


In [15]:
import geopandas as gpd
df = gpd.read_file('e:/ATMRI/CD_R/ATCO_preferences_interface/Scenario_interface_work/sector_bounds/test.geojson')
print(df.head(16))

pop= list(df.geometry[0].exterior.coords)
type(pop)
sec1,sec2,sec3,sec4,sec5,sec6,sec7,sec8 = [],[],[],[],[],[],[],[]

a= df.geometry[0].exterior.coords
for x,y in a:
    sec1.append([x,y])
b= df.geometry[1].exterior.coords
for x,y in b:
    sec2.append([x,y])
c = df.geometry[2].exterior.coords
for x,y in c:
    sec3.append([x,y])
d= df.geometry[3].exterior.coords
for x,y in d:
    sec4.append([x,y])  
e= df.geometry[4].exterior.coords
for x,y in e:
    sec5.append([x,y])
f= df.geometry[5].exterior.coords
for x,y in f:
    sec6.append([x,y]) 
g= df.geometry[6].exterior.coords
for x,y in g:
    sec7.append([x,y])
h= df.geometry[7].exterior.coords
for x,y in h:
    sec8.append([x,y])

d1= {'sector1':sec1}
# for i in range(len(sec1)):
#     d1.setdefault('sector1',[])
#     d1['sector1'].append(sec1[i])

d2= {}
for i in range(len(sec1)):
    d2.setdefault('sector2',[])
    d2['sector2'].append(sec2[i])
d3= {}
for i in range(len(sec3)):
    d3.setdefault('sector3',[])
    d3['sector3'].append(sec3[i])
d4= {}
for i in range(len(sec4)):
    d4.setdefault('sector4',[])
    d4['sector4'].append(sec4[i])
d5= {}
for i in range(len(sec5)):
    d5.setdefault('sector5',[])
    d5['sector5'].append(sec5[i])
d6= {}
for i in range(len(sec6)):
    d6.setdefault('sector6',[])
    d6['sector6'].append(sec6[i])
d7= {}
for i in range(len(sec7)):
    d7.setdefault('sector7',[])
    d7['sector7'].append(sec1[7])
d8= {}
for i in range(len(sec8)):
    d8.setdefault('sector8',[])
    d8['sector8'].append(sec8[i]) 


dd={'Sectors':
    {'sector1':sec1,
    'sector2':sec2,
    'sector3':sec3,
    'sector4':sec4,
    'sector5':sec5,
    'sector6':sec6,
    'sector7':sec7,
    'sector8':sec8}
    }
sectorcoords=[dd]
os.chdir('E:\\atmri\CD_R\ATCO_preferences_interface\Scenario_interface_work\data')
import json
with open('SgSectors_LonLat.json','w') as json_file:
    json.dump(sectorcoords,json_file)

df2=pd.read_json('SgSectors_LonLat.json')


"""the sqaure region with 'a' as the bottom left point (moving anticlock abcd)
    a,b,c,d= [lon,lat] format"""
a = np.array([102,1])
b = np.array([108,1])
c = np.array([108,7])
d = np.array([102,7])

horizontal_length  = b[0]-a[0]
vertical_length = d[1]-a[1]
keylist=['sector1','sector2','sector3','sector4','sector5','sector6','sector7','sector8']
l1=[[],[],[],[],[],[],[],[]]
for i in range(len(keylist)):
    for j in range(len(df2.Sectors[0][keylist[i]])):
        P= np.array([df2.Sectors[0][keylist[i]][j][0],df2.Sectors[0][keylist[i]][j][1]])
        # print(P)
        vert_d = P[1]-a[1]
        horiz_d= P[0]-a[0]
        l1[i].append([(900/horizontal_length)*horiz_d,900-(900/vertical_length)*vert_d])

sectordict = {'Sectors':{
                'sector1':{'type': 'secondary', 'boundary': l1[0]},
                'sector2':{'type': 'secondary', 'boundary': l1[1]},
                'sector3':{'type': 'secondary', 'boundary': l1[2]},
                'sector4':{'type': 'secondary', 'boundary': l1[3]},
                'sector5':{'type': 'secondary',  'boundary': l1[4]},
                'sector6':{'type': 'primary', 'boundary':  l1[5]},
                'sector7':{'type':'secondary', 'boundary': l1[6]},
                'sector8':{'type':'secondary' ,'boundary':l1[7]}
                }}

sectorpoints=[sectordict]
sectorpoints

geometry
0   POLYGON ((104.57350 1.03797, 108.98472 0.15792...
1   POLYGON ((102.16666 1.65000, 103.03467 1.36722...
2   POLYGON ((103.80778 2.00333, 103.78716 2.36750...
3   POLYGON ((104.48297 1.80790, 104.72902 2.61866...
4   POLYGON ((106.57083 3.90445, 109.19967 7.69972...
5   POLYGON ((106.21300 4.28805, 106.57083 3.90445...
6   POLYGON ((103.74417 3.11917, 103.73333 4.83333...
7   POLYGON ((106.57083 3.90445, 106.21300 4.28805...
8   POLYGON ((104.40200 1.54111, 103.98866 1.51972...
9   POLYGON ((102.16666 1.65000, 103.03467 1.36722...
10  POLYGON ((103.84033 1.50389, 103.98866 1.51972...
11  POLYGON ((104.40200 1.54111, 104.72902 2.61866...
12  POLYGON ((103.80778 2.00333, 103.78716 2.36750...
13  POLYGON ((106.57083 3.90445, 106.92616 3.56138...
14  POLYGON ((108.98472 0.15792, 109.00000 0.00000...
15  POLYGON ((110.09216 3.47583, 108.50000 2.25000...


[{'Sectors': {'sector1': {'type': 'secondary',
    'boundary': [[386.02465222603826, 894.3045658388949],
     [1047.7087499999996, 1026.3120833333335],
     [1050.0, 1050.0],
     [898.9995833333339, 1050.0],
     [600.0, 1175.0],
     [474.9995833333337, 1050.0],
     [415.00041666666687, 1050.0],
     [293.49958333333245, 1093.9166666666667],
     [275.5813414167711, 943.8627147460479],
     [278.96781205352636, 944.7157137124241],
     [288.66083759105123, 946.1607735224628],
     [298.44808033342787, 946.6426274200669],
     [308.2353165654773, 946.1565748244218],
     [317.92831124189007, 944.7072402839798],
     [327.4337270938339, 942.3085308407015],
     [336.66002526074476, 938.9835032754531],
     [345.5183485183369, 934.7641425344991],
     [353.92337837859384, 929.691053557577],
     [361.7941576480313, 923.8130696148297],
     [369.0548704374344, 917.1867810998298],
     [375.63557210792237, 909.8759895080681],
     [381.47286220150676, 901.9510920460494],
     [386.024652

In [16]:
df = pd.read_json('E:/atmri/CD_R/ATCO_preferences_interface/Scenario_interface_work/python-code/14Test1.json')


(df.segments[0][1])

{'name': '20DME (20 DME PU)',
 'lat': '1.758',
 'lon': '103.970',
 'latdms': '014530N',
 'londms': '1035812E',
 'transf_x': -6.750000000000256,
 'transf_y': 170.55}

In [17]:
datalist=[]
for i in range(10):

    waypointList=[]
    for i in range(len(df)):
        for j in range(len(df.segments[i])):
            waypointList.append(WayPoint(df.segments[i][j]['name'],[df.segments[i][j]['transf_x'],900-df.segments[i][j]['transf_y']]))

    airwayList=[]
    a=0
    for i in range(len(df.segments)):
        x=len(df.segments[i])
        # print('x:',x)
        for j in range(a,len(waypointList[:a+x])-1):
            print('j: ',j)
            li= [waypointList[j],waypointList[j+1]]
            print(li)
            airwayList.append(Airways('aw'+str(j), li[0],li[1]))
        a+=x
       
    # for k in range(len(waypointList)-1):
    #     li=[waypointList[k],waypointList[k+1]]
    #     airwayList.append(Airways('aw'+str(k), li[0],li[1]))



    ac_per_scenario = 6
    aircraftList=[]
    aircraft_name=[]
    for n in range(1,ac_per_scenario+1):
        aircraft_name.append('ac'+str(n))
    for i in range(ac_per_scenario):
        point=np.random.choice(airwayList)
        d=[20,30,40,50,60,70,80,90,100,110,120,140,150,170,190,210,220,230,240,250] #offset,
        aircraft = Aircraft(str(aircraft_name[i]), point , np.random.choice(d)) #this randomization will create multiple scenaios
        aircraftList.append(aircraft)


    d= Scenario(waypointList,airwayList)
    dict4={}
    for i in range(len(waypointList)):
        waydict={waypointList[i].name:{
            'x':waypointList[i].x,
            'y':waypointList[i].y}}
        dict4.update(waydict)

    dict5={}
    for i in range(len(airwayList)):
        airwaydict= {airwayList[i].name:{
            'start':airwayList[i].start_wp,
            'end' : airwayList[i].end_wp}}
        dict5.update(airwaydict)

    dict6={}
    for i in range(len(aircraftList)):
        aircraftdict= {aircraftList[i].name:{
            'x':aircraftList[i].offset_x,
            'y':aircraftList[i].offset_y,
            'dir_x': aircraftList[i].dir_x,
            'dir_y': aircraftList[i].dir_y}}
        dict6.update(aircraftdict)    

    data_dict={'waypoints':dict4, 'airways':dict5,'aircrafts':dict6, 
        'sectors':{
                    'sector1':{'type': 'secondary', 'boundary': l1[0]},
                    'sector2':{'type': 'secondary', 'boundary': l1[1]},
                    'sector3':{'type': 'secondary', 'boundary': l1[2]},
                    'sector4':{'type': 'secondary', 'boundary': l1[3]},
                    'sector5':{'type': 'secondary',  'boundary': l1[4]},
                    'sector6':{'type': 'primary', 'boundary':  l1[5]},
                    'sector7':{'type':'secondary', 'boundary': l1[6]},
                    'sector8':{'type':'secondary' ,'boundary':l1[7]}
                    }}


    datalist.append(data_dict) 

_Copy1.WayPoint object at 0x000001DC0669FB80>]
j:  115
[<classLib_yash_Copy1.WayPoint object at 0x000001DC0669FB80>, <classLib_yash_Copy1.WayPoint object at 0x000001DC0669FBE0>]
j:  117
[<classLib_yash_Copy1.WayPoint object at 0x000001DC0669FC70>, <classLib_yash_Copy1.WayPoint object at 0x000001DC0669FCA0>]
j:  118
[<classLib_yash_Copy1.WayPoint object at 0x000001DC0669FCA0>, <classLib_yash_Copy1.WayPoint object at 0x000001DC0669FD00>]
j:  119
[<classLib_yash_Copy1.WayPoint object at 0x000001DC0669FD00>, <classLib_yash_Copy1.WayPoint object at 0x000001DC0669FD60>]
j:  120
[<classLib_yash_Copy1.WayPoint object at 0x000001DC0669FD60>, <classLib_yash_Copy1.WayPoint object at 0x000001DC0669FDC0>]
j:  121
[<classLib_yash_Copy1.WayPoint object at 0x000001DC0669FDC0>, <classLib_yash_Copy1.WayPoint object at 0x000001DC0669FE20>]
j:  122
[<classLib_yash_Copy1.WayPoint object at 0x000001DC0669FE20>, <classLib_yash_Copy1.WayPoint object at 0x000001DC0669FEB0>]
j:  124
[<classLib_yash_Copy1.WayPoi

In [20]:

Path.cwd()

WindowsPath('E:/atmri/CD_R/ATCO_preferences_interface/Scenario_interface_work/data')

In [22]:
import json
with open('scenarioWith_sector_allairwaytest_14july.json', 'w') as json_file:
    json.dump(datalist, json_file)
